In [1]:
# import sys
# import pprint
# 
# sys.setrecursionlimit(10000)  # if needed
# pp = pprint.PrettyPrinter(depth=6, width=120)
# # pp.pprint(large_object)

In [2]:
import re

def parse_facts(file_path):
    facts = {}
    target = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith("target:"):
                target_line = line[len("target:"):].strip()
                key, op, val = parse_condition(target_line)
                target = (key, val)  # Only store key and expected value
                continue
            if '=' in line:
                key, val = line.split('=')
                try:
                    facts[key.strip()] = int(val.strip())
                except:
                    facts[key.strip()] = val.strip()
            elif 'is' in line:
                key, val = line.split('is')
                facts[key.strip()] = val.strip()
            else:
                facts[line.strip()] = True
    return facts, target

In [3]:
def parse_condition(cond):
    cond = cond.strip()
    if '>=' in cond:
        key, val = cond.split('>=')
        return (key.strip(), '>=', float(val.strip()))
    elif '<=' in cond:
        key, val = cond.split('<=')
        return (key.strip(), '<=', float(val.strip()))
    if '>' in cond:
        key, val = cond.split('>')
        return (key.strip(), '>', float(val.strip()))
    elif '<' in cond:
        key, val = cond.split('<')
        return (key.strip(), '<', float(val.strip()))
    elif '=' in cond:
        key, val = cond.split('=')
        try:
            return (key.strip(), '=', int(val.strip()))
        except:
            return (key.strip(), '=', val.strip())
    elif 'is' in cond:
        key, val = cond.split('is')
        return (key.strip(), 'is', val.strip())
    else:
        return (cond.strip(), '==', True)


In [4]:
def parse_rules(file_path):
    rules = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or 'IF' not in line or 'THEN' not in line:
                continue
            cond_part, conclusion = line.split('THEN')
            cond_part = cond_part.replace('IF', '').strip()
            conclusion = conclusion.strip()

            and_conditions = [c.strip() for c in re.split(r'\bAND\b', cond_part)]
            condition_list = []
            for cond in and_conditions:
                if 'OR' in cond:
                    or_parts = [c.strip() for c in cond.split('OR')]
                    or_group = [parse_condition(c) for c in or_parts]
                    condition_list.append(('OR', or_group))
                else:
                    condition_list.append(parse_condition(cond))
            conclusion_parsed = parse_condition(conclusion)
            rules.append((condition_list, conclusion_parsed))
    return rules


In [5]:
def evaluate_condition(facts, cond):
    key, op, val = cond
    if key not in facts:
        return False
    try:
        if op == '=' or op == 'is':
            return facts[key] == val
        elif op == '==':
            return bool(facts[key])
        elif op == '>':
            return float(facts[key]) > float(val)
        elif op == '<':
            return float(facts[key]) < float(val)
        elif op == '>=':
            return float(facts[key]) >= float(val)
        elif op == '<=':
            return float(facts[key]) <= float(val)
    except:
        return False
    return False


In [6]:
def evaluate_conditions(facts, conditions):
    for cond in conditions:
        if isinstance(cond, tuple) and cond[0] == 'OR':
            if not any(evaluate_condition(facts, c) for c in cond[1]):
                return False
        else:
            if not evaluate_condition(facts, cond):
                return False
    return True


In [7]:
def forward_chaining(facts, rules, target=None):
    changed = True
    while changed:
        changed = False
        print("Current facts:", facts)
        for conditions, conclusion in rules:
            key, op, val = conclusion
            if key in facts and facts[key] == val:
                continue
            if evaluate_conditions(facts, conditions):
                facts[key] = val
                changed = True
                if target and key == target[0] and facts[key] == target[1]:
                    print("Current facts:", facts)
                    print(f"\nTarget reached: {key} ")
                    return facts
    return facts


In [8]:
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

In [9]:
print("\n--- Forward Chaining ---")
final_facts = forward_chaining(facts, rules, target)

print("\nFinal inferred facts:")
for fact in final_facts:
    print(f"{fact} = {final_facts[fact]}")


--- Forward Chaining ---
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange'}
Current facts: {'seeds': 0, 'diameter': 7, 'skin_smell': True, 'color': 'orange', 'perfumed': True, 'size': 'medium', 'fruit': 'orange', 'citrus_fruit': True}

Target reached: citrus_fruit 

Final inferred facts:
seeds = 0
diameter = 7
skin_smell = True
color = orange
perfumed = True
size = medium
fruit = orange
citrus_fruit = True


In [10]:
def backward_chaining(facts, rules, target, depth=0, visited=None):
    indent = " " * depth
    target_key, target_val = target
    print(f"{indent}Trying to prove: {target_key} = {target_val}")
    
    # Check if target is already in facts
    if target_key in facts:
        result = facts[target_key] == target_val
        print(f"{indent}{'✓' if result else '×'} Known: {target_key} = {facts[target_key]}")
        return result
        
    if visited is None:
        visited = set()
    
    # Prevent infinite recursion
    target_str = f"{target_key}={target_val}"
    if target_str in visited:
        print(f"{indent}× Loop detected: {target_str}")
        return False
    visited.add(target_str)
    
    # Try each relevant rule
    for i, (conditions, conclusion) in enumerate(rules):
        if conclusion[0] == target_key and conclusion[2] == target_val:
            print(f"{indent}Checking rule {i+1}")
            
            # Process each condition
            all_met = True
            temp_facts = facts.copy()
            
            for condition in conditions:
                if isinstance(condition, tuple) and condition[0] == 'OR':
                    # Handle OR conditions
                    or_conditions = condition[1]
                    print(f"{indent} Checking OR condition")
                    or_met = False
                    
                    for or_cond in or_conditions:
                        cond_key, _, cond_val = or_cond
                        if cond_key in temp_facts and evaluate_condition(temp_facts, or_cond):
                            or_met = True
                            break
                        elif backward_chaining(temp_facts, rules, (cond_key, cond_val), depth+1, visited.copy()):
                            or_met = True
                            break
                            
                    if not or_met:
                        all_met = False
                        break
                else:
                    # Handle regular condition
                    cond_key, _, cond_val = condition
                    if cond_key in temp_facts and evaluate_condition(temp_facts, condition):
                        continue
                    elif not backward_chaining(temp_facts, rules, (cond_key, cond_val), depth+1, visited.copy()):
                        all_met = False
                        break
            
            if all_met:
                facts.update(temp_facts)
                facts[conclusion[0]] = conclusion[2]
                print(f"{indent}✓ Rule fired: {conclusion[0]} = {conclusion[2]}")
                return True
    
    print(f"{indent}× Failed to prove: {target_key} = {target_val}")
    return False

In [11]:
# Parse facts and rules using your existing functions
facts, target = parse_facts('/Users/mac/Desktop/FCAI/reasoning/facts.txt')
rules = parse_rules('/Users/mac/Desktop/FCAI/reasoning/rules.txt')

print("\n--- Backward Chaining ---")
result = backward_chaining(facts, rules, (target[0], target[1]))
print(f"\nTarget '{target[0]} = {target[1]}' inferred: {result}")
print("\nFinal facts after inference:")
for key, value in sorted(facts.items()):
    print(f"{key} = {value}")


--- Backward Chaining ---
Trying to prove: citrus_fruit = True
Checking rule 5
 Checking OR condition
 Trying to prove: fruit = lemon
 Checking rule 6
  Trying to prove: size = medium
  Checking rule 13
  ✓ Rule fired: size = medium
  Trying to prove: color = yellow
  × Known: color = orange
 × Failed to prove: fruit = lemon
 Trying to prove: fruit = orange
 Checking rule 9
  Trying to prove: perfumed = True
  Checking rule 4
  ✓ Rule fired: perfumed = True
  Trying to prove: size = medium
  Checking rule 13
  ✓ Rule fired: size = medium
 ✓ Rule fired: fruit = orange
✓ Rule fired: citrus_fruit = True

Target 'citrus_fruit = True' inferred: True

Final facts after inference:
citrus_fruit = True
color = orange
diameter = 7
fruit = orange
perfumed = True
seeds = 0
size = medium
skin_smell = True
